<a href="https://colab.research.google.com/github/GitHub-Bong/Toxic-Comment-Challenge/blob/master/0401_baseline_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow import keras
import sys, os, re, string, csv, codecs, numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers

In [ ]:
train = pd.read_csv('/content/drive/Shareddrives/SOGANG Parrot/train.csv/train.csv')
test = pd.read_csv('/content/drive/Shareddrives/SOGANG Parrot/test.csv/test.csv')

In [ ]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
train.isnull().any()

id               False
comment_text     False
toxic            False
severe_toxic     False
obscene          False
threat           False
insult           False
identity_hate    False
dtype: bool

In [ ]:
test.isnull().any()

id              False
comment_text    False
dtype: bool

In [ ]:
# download the stopwords from NLTK
import nltk                         
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [ ]:
train['comment_text'] = train['comment_text'].apply(lambda x: clean_text(x))

In [ ]:
train['comment_text'].head()

0    explanationwhy the edits made under my usernam...
1    daww he matches this background colour im seem...
2    hey man im really not trying to edit war its j...
3    morei cant make any real suggestions on improv...
4    you sir are my hero any chance you remember wh...
Name: comment_text, dtype: object

In [ ]:
test['comment_text'] = test['comment_text'].apply(lambda x: clean_text(x))

In [ ]:
test['comment_text'].head()

0    yo bitch ja rule is more succesful then youll ...
1            from rfc   the title is fine as it is imo
2              sources    zawe ashton on lapland —    
3    if you have a look back at the source the info...
4              i dont anonymously edit articles at all
Name: comment_text, dtype: object

In [ ]:
from nltk.corpus import stopwords  

def remove_stopwords(text):
    
    words = [word for word in text if word not in stopwords.words('english')]
    return words

In [ ]:
# 꽤 오래걸리는거같아서 일단 대기
# train['comment_text'] = train['comment_text'].apply(lambda x: remove_stopwords(x))
# train.head()

In [ ]:
# 꽤 오래걸리는거같아서 일단 대기
# test['comment_text'] = test['comment_text'].apply(lambda x: remove_stopwords(x))
# test.head()

In [ ]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values # y.shape (159571, 6)
list_sentences_train = train["comment_text"] # (159571,)
list_sentences_test = test["comment_text"] # (153164,)

In [ ]:
list_sentences_train[0]

'explanationwhy the edits made under my username hardcore metallica fan were reverted they werent vandalisms just closure on some gas after i voted at new york dolls fac and please dont remove the template from the talk page since im retired '

In [ ]:
max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
# list_tokenized_train[:1] = [[688,75,1,126,130, ,,, ]]
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
# list_tokenized_test[:1] = [[2665,655,8849,656, ,,, ]]

# tokenizer.word_counts = OrderedDict([('explanation', 1771),('why', 17818),('the', 496540),('edits', 9957), ,,, ])
# tokenizer.word_index = {'the': 1,'to': 2,'of': 3,'and': 4, ,,, }



In [ ]:
maxlen = 200
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen) # (159571, 200)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen) # (153164, 200)

In [ ]:
inp = Input(shape=(maxlen, )) #maxlen=200

In [ ]:
embedding_dict = dict()
f = open('/content/drive/Shareddrives/SOGANG Parrot/glove.twitter.27B.25d.txt/glove.twitter.27B.25d.txt', encoding="utf8")

for line in f:
    word_vector = line.split()
    word = word_vector[0]
    word_vector_arr = np.asarray(word_vector[1:], dtype='float32') # 100
    embedding_dict[word] = word_vector_arr
f.close()
print('%s개의 Embedding vector가 있습니다.' % len(embedding_dict))

1193515개의 Embedding vector가 있습니다.


In [ ]:
print(embedding_dict['respectable'])
print(len(embedding_dict['respectable']))

[-0.49053  -0.43373  -1.4828    0.07678   0.77266   1.0036   -0.16272
 -1.2494   -0.13092  -1.03     -1.0419    0.35045  -1.9792    0.53035
 -0.070342  0.4989    0.85868  -0.080579  0.13071   0.40617   0.68475
  0.7763   -0.98371  -0.89656  -1.1979  ]
25


In [ ]:
embedding_matrix = np.zeros((len(tokenizer.word_index)+1, 25)) # will delete first row
np.shape(embedding_matrix)

(268592, 25)

In [ ]:
for word, i in tokenizer.word_index.items(): 
    temp = embedding_dict.get(word) 
    if temp is not None:
        embedding_matrix[i] = temp 

In [ ]:
embedding_matrix.shape

(268592, 25)

In [ ]:
embedding_matrix.shape[1]

25

In [ ]:
embedding_matrix = np.delete(embedding_matrix,0,axis=0) # delete first row
embedding_matrix.shape

(268591, 25)

In [ ]:
np.save('/content/drive/Shareddrives/SOGANG Parrot/0401pretrained-embed-Glove.npy',embedding_matrix)

In [ ]:
x = Embedding(len(tokenizer.word_index), embedding_matrix.shape[1],
              weights=[embedding_matrix],trainable=False)(inp)

In [ ]:
x = Bidirectional(LSTM(60, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)

In [ ]:
x = GlobalMaxPool1D()(x)

In [ ]:
x = Dropout(0.1)(x)

In [ ]:
x = Dense(50, activation="relu")(x)

In [ ]:
x = Dropout(0.1)(x)

In [ ]:
x = Dense(6, activation="sigmoid")(x)

In [ ]:
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [ ]:
inp = Input(shape=(maxlen, )) #maxlen=200
x = Embedding(len(tokenizer.word_index), embedding_matrix.shape[1],
              weights=[embedding_matrix],trainable=False)(inp)
x = Bidirectional(LSTM(60, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.1)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 200, 25)           6714775   
_________________________________________________________________
bidirectional (Bidirectional (None, 200, 120)          41280     
_________________________________________________________________
global_max_pooling1d (Global (None, 120)               0         
_________________________________________________________________
dropout (Dropout)            (None, 120)               0         
_________________________________________________________________
dense (Dense)                (None, 50)                6050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0   

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#checkpoint
checkpoint = ModelCheckpoint(filepath='/content/drive/Shareddrives/SOGANG Parrot/0401pretrained-embed-Glove.hdf5', monitor='val_loss', verbose=1, save_best_only=True)

batch_size = 512
epochs = 2
hist = model.fit(X_t,y, batch_size=batch_size, epochs=epochs, callbacks=[checkpoint], validation_split=0.1)

Epoch 1/2
281/281 [==============================] - 362s 1s/step - loss: 0.1422 - accuracy: 0.7720 - val_loss: 0.1180 - val_accuracy: 0.9937

Epoch 00001: val_loss improved from inf to 0.11802, saving model to /content/drive/Shareddrives/SOGANG Parrot/0401pretrained-embed-Glove.hdf5
Epoch 2/2
281/281 [==============================] - 356s 1s/step - loss: 0.1148 - accuracy: 0.9328 - val_loss: 0.1008 - val_accuracy: 0.9930

Epoch 00002: val_loss improved from 0.11802 to 0.10082, saving model to /content/drive/Shareddrives/SOGANG Parrot/0401pretrained-embed-Glove.hdf5


In [ ]:
y_test = model.predict(X_te)

In [ ]:
sample_submission = pd.read_csv("/content/drive/Shareddrives/SOGANG Parrot/sample_submission.csv/sample_submission.csv")

sample_submission[list_classes] = y_test

sample_submission.to_csv("/content/drive/Shareddrives/SOGANG Parrot/baseline/20210401-pretrained-embed-Glove.csv", index=False)

In [ ]:
sample_submission.head(15)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.234336,0.016442,0.123655,0.008953,0.130799,0.025306
1,0000247867823ef7,0.062597,0.003755,0.027164,0.002575,0.020276,0.006350
2,00013b17ad220c46,0.116728,0.011081,0.061193,0.006957,0.052779,0.014141
3,00017563c3f7919a,0.031171,0.000889,0.012010,0.000900,0.010256,0.002672
4,00017695ad8997eb,0.115962,0.009738,0.061381,0.005041,0.046592,0.008789
5,0001ea8717f6de06,0.090968,0.005127,0.042159,0.003140,0.032600,0.007766
6,00024115d4cbde0f,0.101697,0.005157,0.045055,0.004508,0.038720,0.009195
7,000247e83dcc1211,0.154305,0.015847,0.074876,0.011851,0.069499,0.025356
8,00025358d4737918,0.022399,0.000581,0.008455,0.000540,0.006871,0.001771
9,00026d1092fe71cc,0.013929,0.000335,0.004622,0.000339,0.003471,0.000827


In [ ]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])